In [1]:
import numpy as np

# Données du réseau

Rd_ij : Partie réelle de l’impédance Zij du réseau (Régime Direct) vu de la ligne.

Xd_ij : Partie imaginaire de l’impédance Zij du réseau (Régime Direct) vu de la ligne.

Rh_ij : Partie réelle de l’impédance Zij du réseau (Régime Homopolaire) vu de la ligne.

Xh_ij : Partie imaginaire de l’impédance Zij du réseau (Régime Homopolaire) vu de la ligne.

Dont (i,j) $\in$ {(1,1),(1,2), (2,1), (2,2)}

donnee_225 : 1 si donnees du reperees au reseau 225kV, 0 cas écheant.

In [2]:
donnees_225 = 1 # 
Rd_11 = 62.50
Xd_11 = 217.35
Rd_12 = 20.54
Xd_12 = 52.30
Rd_22 = 2.68
Xd_22 = 23.60
Rh_11 = 149.11
Xh_11 = 1040.56
Rh_12 = 42.73
Xh_12 = 182.40
Rh_22 = 13.78
Xh_22 = 188.52

# Données de ligne

Rd : Partie réelle de l’impédance Zl de ligne (Régime Direct).

Xd : Partie imaginaire de l’impédance Zl de ligne (Régime Direct).

Rh : Partie réelle de l’impédance Zl de ligne (Régime Homopolaire).

Xh : Partie imaginaire de l’impédance Zl de ligne (Régime Homopolaire).

L : Longuer de la ligne [Km]

In [3]:
# Donnees de ligne
Rd = 25.13
Xd = 64.03
Rh = 48.47
Xh = 228.32
L = 12.60 # Km


# Données "Generiques"

Xpyl : Position relative du support

tension_code_LIT : 3 si réseau 63kV, 4 si réseau 90kV, 6 si réseau 225kV, 7 si réseau 400kV

t_elim : Temps d'élimination du défaut [s]

In [4]:
x_pyl = 0.0976 
tension_code_LIT = 3
t_elim = 500e-3

# Données "de Terre"

rho : Resistivité du terrain [$\Omega \cdot $m]

long_empat : Longueur de l'empatement [m]

larg_empat : Largeur de l'empatement [m]

In [5]:
rho = 400 # Ohm * m
long_empat = 2.60
larg_empat = 2.60
#Re = 1.67 # m

Calcul du coefficient de redressement de tension (Si données d'impedances reperées au réseau 225kV, i.e: donnees_225 = 1)

In [6]:
if donnees_225 == 1:
    if tension_code_LIT == 2:
        U = 49.0 #Reseau 45kV
        Cr = (45.0 / 225.0)**2
    elif tension_code_LIT == 3:
        U = 69.0 #Reseau 63kV
        Cr = (63.0 / 225.0)**2
    elif tension_code_LIT == 4:
        U = 99.0 #Reseau 90kV
        Cr = (90.0 / 225.0)**2
    elif tension_code_LIT == 5:
        U = 165.0 #Reseau 150kV
        Cr = (150.0 / 225.0)**2
    elif tension_code_LIT == 6:
        U = 245.0 #Reseau 225kV
        Cr = 1.0
    elif tension_code_LIT == 7:
        U = 420.0 #Reseau 400kV
        Cr = (400.0 / 225.0)**2
else:
    Cr = 1.0
    if tension_code_LIT == 2:
        U = 49.0 #Reseau 45kV
    elif tension_code_LIT == 3:
        U = 69.0 #Reseau 63kV
    elif tension_code_LIT == 4:
        U = 99.0 #Reseau 90kV
    elif tension_code_LIT == 5:
        U = 165.0 #Reseau 150kV
    elif tension_code_LIT == 6:
        U = 245.0 #Reseau 225kV
    elif tension_code_LIT == 7:
        U = 420.0 #Reseau 400kV

# Calcul d'impedances en directe

$$Z_d = C_r \left[ x (1-x) (Z_{d,L}-Z_{d,12})+(Z_{d,11}-x  Z_{d,12})\frac{(1-x)Z_{d,12}+Z_{d,22}}{Z_{d,11}+Z_{d,12}+Z_{d,22}} \right]$$

In [7]:
ZdL = Rd + 1j*Xd
Zd11 = Rd_11 + 1j*Xd_11
Zd12 = Rd_12 + 1j*Xd_12
Zd22 = Rd_22 + 1j*Xd_22
Zd = Cr * ( x_pyl * ( 1 - x_pyl ) * ( ZdL - Zd12 )
           + ( Zd11 + x_pyl * Zd12 ) * ( ( ( 1.0 - x_pyl ) * Zd12 + Zd22 ) / ( Zd11
           + Zd12 + Zd22 ) ) )

# Calcul d'impedances en homopolaire

$$Z_h = C_r \left[ x (1-x) (Z_{h,L}-Z_{h,12})+(Z_{h,11}-x  Z_{h,12})\frac{(1-x)Z_{h,12}+Z_{h,22}}{Z_{h,11}+Z_{h,12}+Z_{h,22}} \right]$$

In [8]:
ZhL = Rh + 1j*Xh
Zh11 = Rh_11 + 1j*Xh_11
Zh12 = Rh_12 + 1j*Xh_12
Zh22 = Rh_22 + 1j*Xh_22
Zh = Cr * ( x_pyl * ( 1 - x_pyl ) * ( ZhL - Zh12 )
           + ( Zh11 + x_pyl * Zh12 ) * ( ( ( 1.0 - x_pyl ) * Zh12 + Zh22 ) / ( Zh11
           + Zh12 + Zh22 ) ) )

# Calcul des courants de CC

$I_{T,h}$: Courant total (apport des postes 1 et poste 2) homopolaire (et égal au courant total direct) sur défaut monophasé

$$ I_{T,h} = \frac{1}{3} \frac{\sqrt{3} U}{2 Z_d + Z_h}$$

$I_{h,A1}$: Apport de courant homopolaire coté poste A dans une configuration étoile sur un défaut monophasé phase-terre, pour une résistance de défaut nulle.

$$ I_{h,A1} = I_{T,h} \frac{Z_{h,22} + ( 1 - x ) Z_{h,12}}{Z_{h,11} + Z_{h,22} + Z_{h,12}} $$

$I_{h,B1}$: Apport de courant homopolaire coté poste A dans une configuration étoile sur un défaut monophasé phase-terre, pour une résistance de défaut nulle

$$ I_{h,B1} = I_{T,h} \frac{Z_{h,11} + x Z_{h,12}}{Z_{h,11} + Z_{h,22} + Z_{h,12}} $$

$I_{h,A}$: Apport de courant homopolaire coté poste A dans une configuration triangle sur un défaut monophasé phase-terre, pour une résistance de défaut nulle

$$ I_{h,A} = \frac{\frac{Z_h I_{T,h}}{C_r} - Z_{h,11} I_{h,A1}}{x Z_{h,L}} $$

$I_{h,B}$: Apport de courant homopolaire coté poste B dans une configuration triangle sur un défaut monophasé phase-terre, pour une résistance de défaut nulle

$$ I_{h,B} = \frac{\frac{Z_h I_{T,h}}{C_r} - Z_{h,22} I_{h,B1}}{(1-x) Z_{h,L}} $$

$Icc_{0,A}$ : Courant provenant du réseau coté poste A lors d’un défaut monophasé phase-terre, pour une résistance de défaut nulle (courant résiduel) 

$$ Icc_{0,A} = 3 \|I_{h,A}\| $$

$Icc_{0,B}$ : Courant provenant du réseau coté poste B lors d’un défaut monophasé phase-terre, pour une résistance de défaut nulle (courant résiduel) 

$$ Icc_{0,B} = 3 \|I_{h,B}\| $$

$Icc_{0,sscdg}$: Courant total provenant du réseau lors d’un défaut phase-terre, pour une résistance de défaut nulle, sans prise en compte de l’effet réducteur des câbles de garde

$$ Icc_{0,sscdg} = Icc_{0,A} + Icc_{0,B} $$


In [9]:
Ith = (1.0 / 3.0) * ( U * np.sqrt(3.0) / (2.0 * Zd + Zh))
IhA1 = Ith * ( Zh22 + ( 1 - x_pyl ) * Zh12 ) / ( Zh11 + Zh22 + Zh12 )
IhB1 = Ith * ( Zh11 + x_pyl * Zh12 ) / ( Zh11 + Zh22 + Zh12 )
IhA = ( ( 1 / Cr ) * Zh * Ith - Zh11 * IhA1 ) / ( x_pyl * ZhL )
IhB = ( ( 1 / Cr ) * Zh * Ith - Zh22 * IhB1 ) / ( ( 1 - x_pyl ) * ZhL )
Icc0_A = 3*abs(IhA)
Icc0_B = 3*abs(IhB)
Icc0_sscdg = (Icc0_A + Icc0_B) * 1e3

# Calcul Resistence de Prise de Terre de Pylone
$D_g$ : demie diagonale de l'empatement [m]

$$ D_g = 0.5 \sqrt{long_{empat}^2 + larg_{empat}^2}$$

$R_e$ : Rayon équivalente [m]

Pour un support en ancrage 63kV/90kV avec 4 malts
$$ R_e = 0,714 D_g + 0,36 $$

$R_p$: Resistence de prise de Terre de pylone [$\Omega$]

$$ R_p = \frac{\rho}{2 \pi R_e} $$

In [10]:
dg = 0.5 * np.sqrt(long_empat**2 + larg_empat**2)
# Re = 0.632 * dg + 0.4
Re = 0.714 * dg + 0.36 #support ancrage
Rp = rho / (2.0 * np.pi * Re)

# Calcul tension de support

$Kr_A$ et $Kr_B$ :Coefficients réducteur du câble de garde coté poste A et B respectivement

$ \| Icc_0 \| $: Courant total provenant du réseau lors d’un défaut phase-terre, pour une résistance de défaut nulle.

$$ \| Icc_0 \| = \| Icc_{0,A} \| Kr_A + \| Icc_{0,A} \| Kr_B $$

$Z_t$ : Impédance totale prenant en compte la prise de terre du pylône et les impédances des câbles de gardes encadrant  

$$ Z_t = \frac{R_p Z_{cdg,A} Z_{cdg,B}}{R_p Z_{cdg,A} + R_p Z_{cdg,B} + Z_{cdg,A} Z_{cdg,B}} $$

$V_{pyl}$ Montée en potentiel du pylône lors d’un défaut monophasé phase-terre, en prenant en compte la résistance du support

$$ V_{pyl} = \frac{U}{\sqrt{3}}\frac{1}{\sqrt{1+\frac{U^2}{3 \| Zt \|^2 \| Icc_0 \| ^2}}}$$


In [11]:
Kr_A = 1.0 # Pour l'instant, pas d'effet du CdG
Kr_B = 1.0 # Pour l'instant, pas d'effet du CdG
Icc0 = Icc0_A * Kr_A + Icc0_B * Kr_B
Zcdg_A = 9999999999 # Pour l'instant, pas d'effet du CdG
Zcdg_B = 9999999999 # Pour l'instant, pas d'effet du CdG
Zt = (Rp * Zcdg_A * Zcdg_B) / (Rp * Zcdg_A + Rp * Zcdg_B + Zcdg_A * Zcdg_B)
Vpyl = ( U / np.sqrt(3) ) / np.sqrt( 1 + ( U**2 / (3*np.abs(Zt)**2 * np.abs(Icc0)**2)))

Pour V $in$ $\{ 650, 1500, 5000 \}$:

$D_{lim, V}$ : Rayon de la zone circulaire, centrée sur le pylône,  pour laquelle la montée en potentielle dépasse V lors d’un défaut.

$$ D_{lim, V} = \frac{R_e V_{pyl}}{V} $$


In [12]:
dlim_5000 = (Re * Vpyl * 1000 )/ (5000.0)
dlim_1500 = (Re * Vpyl * 1000 )/ (1500.0)
dlim_650 = (Re * Vpyl * 1000 )/ (650.0)

In [13]:
print("|IhA1| = {:.3f}".format(abs(IhA1)))
print("|IhB1| = {:.3f}".format(abs(IhB1)))
print("|Icc0| = {:.3f}".format(abs(Icc0)))
print("Rp = {:.3f}".format(Rp))
print("|Zt| = {:.3f}".format(abs(Zt)))
print("|Vp| = {:.3f}".format(abs(Vpyl)))
print("Dlim_5000m = {:.3f}".format(dlim_5000))
print("Dlim_1500m = {:.3f}".format(dlim_1500))
print("Dlim_650m = {:.3f}".format(dlim_650))

|IhA1| = 0.330
|IhB1| = 0.989
|Icc0| = 3.959
Rp = 38.060
|Zt| = 38.060
|Vp| = 38.514
Dlim_5000m = 12.884
Dlim_1500m = 42.947
Dlim_650m = 99.109
